cd luca/Scrivania/Magistrale/SistemiDigitali/Progetto/Project_SistemiDigitali/AES_Encryption/Vivado/AES_CTR_Pynq/

scp design_1.bit xilinx@192.168.2.99:/home/xilinx/pynq/overlays/AES_Encryption/

scp design_1.tcl xilinx@192.168.2.99:/home/xilinx/pynq/overlays/AES_Encryption/

scp design_1.hwh xilinx@192.168.2.99:/home/xilinx/pynq/overlays/AES_Encryption/

In [ ]:
from pynq import Overlay
import pynq.lib.dma
import numpy as np
import time

# Caricamento dell'overlay
overlay = Overlay('/home/xilinx/pynq/overlays/AES_Encryption/design_1.bit')

# Caricamento del DMA
dma = overlay.axi_dma_0

# Check DMA status
sendstatus = dma.sendchannel.running
recvstatus = dma.recvchannel.running
print("sendstatus:", sendstatus," -  recvstatus:",recvstatus)

# Alloca memoria per l'input e l'output -> 2 array da 16 byte (uint8)
data_size = 16
in_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)
out_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)


# Plaintext di prova
plaintext = b"\x04\xb4\x56\xee\x17\xa6\x6a\xbe\x53\x28\xa7\x73\xd6\x7c\xd8\x6a"
plaintext_array = np.array(bytearray(plaintext))
print("Plaintext: ",plaintext)

# Copia dei dati nel buffer
np.copyto(in_buffer, plaintext_array)

# Avviare il trasferimento del DMA ed aspettare i risultati
start = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
end = time.time()

print("Execution time:", end - start, "s")

output = bytearray(out_buffer)
print("Cipher:", output.hex())
#print("Cipher:", bytes(output))

# Dovrebbe venire
#       a9 99 07 o    e3 f3 bf X    a4 b2 b0 [    e5 | g a4
#       a9 99 07 6f   e3 f3 bf 58   a4 b2 b0 5b   e5 7c 67 a4 

print("")
## 2 Iterazione
plaintext = b"\x33\xdd\x56\xee\x17\xa6\x1a\x13\x32\x23\xa7\x73\x65\x1c\xd2\x41"
plaintext_array = np.array(bytearray(plaintext))
print("Plaintext: ",plaintext)
np.copyto(in_buffer, plaintext_array)
start = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
end = time.time()
print("Execution time:", end - start, "s")
output = bytearray(out_buffer)
print("Cipher:", output.hex())


print("")
## 3 Iterazione
plaintext = b"\xc4\x74\x15\x74\xa7\xa6\xc1\xa3\x73\x28\x17\x76\x73\xbc\x3f\xe1"
plaintext_array = np.array(bytearray(plaintext))
print("Plaintext: ",plaintext)
np.copyto(in_buffer, plaintext_array)
start = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
end = time.time()
print("Execution time:", end - start, "s")
output = bytearray(out_buffer)
print("Cipher:", output.hex())


# Liberare i buffer
in_buffer.close()
out_buffer.close()



Il codice di cifratura di singoli blocchi con ECB CORRISPONDE con il risultato ottenuto con src/pyScript/ECB.py.
Ovvero singoli blocchi di 128 bit sono cifrati nel modo GIUSTO.

Unico problema -> la cifratura di un singolo blocco di 128 bit impiega circa 0,002 secondi. Per migliorare le performance occorre parallelizzare il codice dell'IPCore

In [1]:
from pynq import Overlay
import pynq.lib.dma
import numpy as np
import time
from PIL import Image

# Caricamento dell'overlay
overlay = Overlay('/home/xilinx/pynq/overlays/AES_Encryption/design_1.bit')

# Caricamento del DMA
dma = overlay.axi_dma_0

# Check DMA status
sendstatus = dma.sendchannel.running
recvstatus = dma.recvchannel.running
print("sendstatus:", sendstatus," -  recvstatus:",recvstatus)

# Alloca memoria per l'input e l'output -> 2 array da 16 byte (uint8)
data_size = 16
in_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)
out_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)

# Carico l'immagine
original_image = Image.open("image8x6pixel.jpg")
#display(original_image)

# Ottieni i dati dell'immagine come byte
image_data = original_image.tobytes()
# Crea un bytearray dalla data dell'immagine
plaintext = bytearray(image_data)

########
# ATTENZIONE: se l'immagine non ha len multipla di 16 bisogna fare il padding

encrypted_img_bytes = b""

start = time.time()
for i in range(0, len(plaintext), 16):
    chunk = plaintext[i:i+16]

    # Copia dei dati nel buffer
    np.copyto(in_buffer, chunk)

    # Avviare il trasferimento del DMA ed aspettare i risultati
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    encrypted_img_bytes = encrypted_img_bytes + bytes(bytearray(out_buffer))
    
    print("PLAIN:\t",chunk)
    print("CHIPER:\t",bytes(bytearray(out_buffer)))
    print("")
   

end = time.time()

print("Execution time:", end - start, "s")

encrypted_image = Image.frombytes('RGB',(8, 6 ), encrypted_img_bytes) # return an Image object

# Mostro l'immagine
display(encrypted_image)

##########
# Salvare l'immagine su disco causa una perdita di informazioni e rende la decifratura impossibile
# Non si può salvare un'immagine senza estensione

# Salvo l'immagine su disco
# encrypted_image.save("encrypted_image")
with open("file.bin", "wb") as file:
    file.write(encrypted_img_bytes)



/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:691: UserWarning: Interrupt s2mm_introut not created: Could not find UIO device for interrupt pin for IRQ number 62
  interrupt, str(e)))


sendstatus: True  -  recvstatus: True
PLAIN:	 bytearray(b'Ay\xb4Dy\xafKw\xa6R}\xa77b\x8d4')
CHIPER:	 b'B\xc6\x8a\x1d\x98\xc2\x9d&+\x07\x1d0{)M\x90'

PLAIN:	 bytearray(b'a\x8b:e\x905`\x8bO{\xa2p\x98\xbb_\x81')
CHIPER:	 b'\x0e\x13W\xe1\x8a\x81IG2\x9a\x9ejh\xdf"\\'

PLAIN:	 bytearray(b'\x9c\x99\xb8\xcd\xde\xff\xff\x81\xa3\xbc\x97\xb9\xd4\x8f\xb1\xcc')
CHIPER:	 b'\xfa\xc3\xbf\xea\x18\xd4&j\xf8\x95\xf5\x0e\xf9\xdb\xa1\xc8'

PLAIN:	 bytearray(b'\x96\xaa\xa8\x7f\x90\x88\x8a\x95\x85?H7nzl\x8c')
CHIPER:	 b'\x1c;\x94\\\x82\xcdF\xb1\xde\xb1$|r\xed/<'

PLAIN:	 bytearray(b'\x99\x8f\xbd\xcd\xc3\xa2\xb3\xabnq`\xac\xae\x99\xa3\x9e')
CHIPER:	 b'/\xc3\xa24\x90L\x17\xef9\xfa\xb0\n\xbc\xd9PB'

PLAIN:	 bytearray(b'\x81\xac\xa5\x88\xa3\x9f\x84\x8b\x8bs\x97\x9a\x85\x90\x95\x81')
CHIPER:	 b'\x18 \xbb\x05\x97\xe8\xf5(~\xc3\x16dU\x92\x94\xa7'

PLAIN:	 bytearray(b'\xbe\xb9\xb6\xb1\xaa\xa4eWJ\xa9\x99\x89\xbf\xb3\xa5t')
CHIPER:	 b"\xdcV\x82\xdby#'\r`\xad\xba\x0f\xa6a\xbe?"

PLAIN:	 bytearray(b'la\xae\xaa\xa1\x88\x

Cifratura di un'immagine e invio del cifrato in un pacchetto UDP

In [11]:
from pynq import Overlay
import pynq.lib.dma
import numpy as np
import time
from PIL import Image
import socket

#####################################

UDP_IP = "192.168.2.1"
UDP_PORT = 5555

IMAGE_TO_ENCRYPT = "image40x30.jpg"

#####################################

# Caricamento dell'overlay
overlay = Overlay('/home/xilinx/pynq/overlays/AES_Encryption/design_1.bit')

# Caricamento del DMA
dma = overlay.axi_dma_0

# Check DMA status
sendstatus = dma.sendchannel.running
recvstatus = dma.recvchannel.running
print("sendstatus:", sendstatus," -  recvstatus:",recvstatus)

# Alloca memoria per l'input e l'output -> 2 array da 16 byte (uint8)
data_size = 16
in_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)
out_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)

# Carico l'immagine
original_image = Image.open(IMAGE_TO_ENCRYPT)

# Ottieni i dati dell'immagine come byte
image_data = original_image.tobytes()
# Crea un bytearray dalla data dell'immagine
plaintext = bytearray(image_data)

########
# ATTENZIONE: se l'immagine non ha len multipla di 16 bisogna fare il padding

encrypted_img_bytes = b""

start = time.time()
for i in range(0, len(plaintext), 16):
    chunk = plaintext[i:i+16]

    # Copia dei dati nel buffer
    np.copyto(in_buffer, chunk)

    # Avviare il trasferimento del DMA ed aspettare i risultati
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    encrypted_img_bytes = encrypted_img_bytes + bytes(bytearray(out_buffer))
   
end = time.time()

print("Execution time:", end - start, "s")

#encrypted_image = Image.frombytes('RGB',(8, 6 ), encrypted_img_bytes) # return an Image object
# Mostro l'immagine
#display(encrypted_image)

# Invio del cifrato in un pacchetto UDP
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
sock.sendto(encrypted_img_bytes, (UDP_IP, UDP_PORT))
print("Immagine inviata")



sendstatus: True  -  recvstatus: True
Execution time: 0.16717195510864258 s
Immagine inviata


/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:691: UserWarning: Interrupt s2mm_introut not created: Could not find UIO device for interrupt pin for IRQ number 62
  interrupt, str(e)))
